# Yolov5 Pytorch
This notebook shows how to compile Yolov5/Pytorch to AWS Inferentia (inf1 instances) using NeuronSDK.

**Reference:** https://pytorch.org/hub/ultralytics_yolov5/

### Install dependencies

In [ ]:
# Set Pip repository  to point to the Neuron repository
%pip config set global.extra-index-url https://pip.repos.neuron.amazonaws.com
# now restart the kernel

In [ ]:
%pip install -r https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt  # install dependencies

In [ ]:
#Install Neuron PyTorch
%pip install -U torch-neuron==1.10.1.2.2.0.0 neuron-cc[tensorflow] "protobuf<4" torchvision
# use --force-reinstall if you're facing some issues while loading the modules
# now restart the kernel again

### Now load a pre-trained model from model zoo and compile it

In [ ]:
import torch
import torch_neuron
from torchvision import models

model_type='l'
assert(model_type in ['n', 's', 'm', 'l', 'x'])
# Model
model = torch.hub.load('ultralytics/yolov5', f'yolov5{model_type}', pretrained=True)
model.eval()

x = torch.rand([1, 3, 640, 640], dtype=torch.float32)
y = model(x)

traced_model = torch.jit.trace(model, x) # check if it is traceable

### Analyze it to see if it is compatible with NeuronSDK

In [ ]:
torch.neuron.analyze_model(model, example_inputs=x)

### Now, compile the model for Inferentia

In [ ]:
model_neuron = torch.neuron.trace(
    model, 
    example_inputs=x,
    skip_compiler=False, 
    dynamic_batch_size=False,
    verbose='debug',
    compiler_workdir="logs/"
)

## Export to saved model
model_neuron.save("yolov5_neuron.pt")

### A quick test

In [ ]:
y = model_neuron(x)
%timeit model_neuron(x)

### Finally, a test with a real image
This code computes the bounding boxes+classes+confidence of each object. These functions need to be kept outside the model to keep the compatibility with NeuronSDK.

In [ ]:
%matplotlib inline
import sys
if '..' not in sys.path: sys.path.append('..')
import torch
import torch.neuron
import matplotlib.pyplot as plt
from common import processing

img = processing.load_sample_imgB()
model_neuron = torch.load('yolov5_neuron.pt')

x = processing.preprocess_coco(img)
x = torch.from_numpy(x)

y = model_neuron(x)
out_img = processing.postprocess_yolov5(y, img)

plt.figure(figsize=(10,10))
plt.imshow(out_img)